In [5]:
from neo4j import GraphDatabase

uri = "neo4j://localhost:7687"
auth = ("neo4j", "testtest")

with GraphDatabase.driver(uri, auth=auth) as driver:
    driver.verify_connectivity()

In [ ]:
# write some data

from uuid import uuid4

def add_author_tx(tx, id, name):
    qry = """
    MERGE (a:Author {
        authorId: $id,
        name: $name
    })
    """
    return tx.run(qry, id=id, name=name)

def add_book_tx(tx, id, title, pages):
    qry = """
    MERGE (b:Book {
        bookId: $id,
        title: $title,
        pages: $pages
    })
    """
    return tx.run(qry, id=id, title=title, pages=pages)

def add_book(tx, title, author, pages):
    bookId = str(uuid4())
    authorId = str(uuid4())
    add_book_tx(tx, id=bookId, title=title, pages=pages)
    add_author_tx(tx, id=authorId, name=author)

    res = tx.run("""
    MATCH (a:Author {authorId:$authorId})
    MATCH (b:Book {bookId:$bookId})
    MERGE (a)-[r:WROTE]->(b)
    """, authorId=authorId, bookId=bookId)
    return res.single()

with GraphDatabase.driver(uri, auth=auth) as driver:
    with driver.session(database="neo4j") as session:
        title = "1Q84"
        author = "Haruki Murakami"
        pages = 900
        session.execute_write(add_book, title, author, pages)



None


In [17]:
# query data

def who_wrote(tx, title):
    qry = """
    MATCH (a:Author)-[r:WROTE]->(b:Book {title:$title})
    RETURN a.name as Author, b.title as Book;
    """
    res = tx.run(qry, title)
    return res.single()

with GraphDatabase.driver(uri, auth=auth) as driver:
    with driver.session(database="neo4j") as session:
        res = session.execute_read(who_wrote, {"title": "1Q84"})
        print(res)

<Record Author='Haruki Murakami' Book='1Q84'>
